In [4]:
import numpy as np

In [23]:
def my_feature_sign(A, y, threshold):
    i = 1
    j = 0
    tmp = np.dot(A.T, A)
    theta = np.zeros(tmp.shape[0], dtype=np.int8)
    x = np.zeros(tmp.shape[0])
    ATy = np.dot(A.T, y)
    grad = -2 * ATy
    now_active = set()
    y2 = np.dot(y.T, y)
    
    
    while i+j:
        if j<1:
            tmpx = np.argmax(np.abs(grad)*(theta==0))
            if grad[tmpx] > threshold:
                theta[tmpx] = -1
                now_active.add(tmpx)
            elif grad[tmpx] < -threshold:
                theta[tmpx] = 1
                now_active.add(tmpx)
            
        index = np.array(sorted(now_active))
        tmp_H = tmp[np.ix_(index, index)]
        tmp_Ay = ATy[index]
        thetaHat = theta[index]
        b = tmp_Ay-(threshold*thetaHat)/2
        x_H = np.linalg.solve(np.atleast_2d(tmp_H), b)
        thetaNew = np.sign(x_H)
        oldx = x[index]
        changed = np.where(abs(thetaNew - thetaHat) > 1)[0]
        if len(changed) > 0:
            xb = x_H
            Ax2 = np.dot(x_H, np.dot(tmp_Ay,x_H))
            ab = np.dot(x_H, tmp_Ay)
            obj = (y2 + Ax2- 2 * ab + threshold*abs(x_H).sum())
            for elem in changed:
                x = x_H[elem]
                y = oldx[elem]
                prop = y / (y - x)
                now = oldx - prop * (oldx - x_H)
                tmp1 = np.dot(now, np.dot(tmp_H, now))
                tmp2 = np.dot(now, tmp_Ay)
                costnow = y2 + (tmp1-2*tmp2+ threshold*abs(now).sum())
                if costnow < obj:
                    obj = costnow
                    best_prop = prop
                    xb = now
        else:
            xb = x_H
        x[index] = xb
        index_zeros = index[np.abs(x[index]) < 1e-18]
        x[index_zeros] = 0.
        theta[index] = np.int8(np.sign(x[index]))
        now_active.difference_update(index_zeros)
        grad = - 2 * ATy + 2 * np.dot(tmp, x)
        tmp_res = np.max(abs(grad[theta == 0]))
        #tmp_res=np.inf
        ntmp_res = np.max(abs(grad[theta != 0] + threshold * theta[theta != 0]))
        
        if tmp_res > threshold:
            i=1
        else:
            i=0
            
        if not np.allclose(ntmp_res,0):
            j=1
        else:
            j=0
    return x

In [26]:
## How to use ?
## Call the function as my_feature_sign(a,y,gamma)
## a is matrix, y is a 1-D array and gamma is a scalar value.
## it returns the result x which is a 1d array.

